In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
%matplotlib inline
from datetime import datetime, timedelta

In [ ]:
name = 'AlexLyzhov'
extract_path = "/Users/alexlyzhov/Documents/data/sleep/MyFitbitData"

In [ ]:
import zipfile
import os
name_folder = os.path.join(extract_path, name)
name_contents = os.listdir(name_folder)
sleep_folder = os.path.join(name_folder, "Sleep")
sleep_contents = os.listdir(sleep_folder)

import json

# Get all json files for sleep data
sleep_files = [file for file in sleep_contents if file.startswith('sleep-') and file.endswith('.json')]

# Sort the files to start from the earliest
sleep_files.sort()

# Load the first JSON file to examine its structure
with open(os.path.join(sleep_folder, sleep_files[0]), 'r') as file:
    sample_data = json.load(file)

In [ ]:
# Initialize lists to store sleep times for 2021, 2022 and 2023
sleep_times_2021 = []
sleep_times_2022 = []
sleep_times_2023 = []

# Loop over all the sleep json files
for sleep_file in sleep_files:
    with open(os.path.join(sleep_folder, sleep_file), 'r') as file:
        data = json.load(file)
        
        # Loop over all sleep records in the file
        for record in data:
            # Extract the start and end times
            start_time = datetime.strptime(record['startTime'], "%Y-%m-%dT%H:%M:%S.%f")
            end_time = datetime.strptime(record['endTime'], "%Y-%m-%dT%H:%M:%S.%f")

            # Calculate sleep duration
            sleep_duration = (end_time - start_time).total_seconds() / 3600
            
            # Extract sleep records for the year 2021, 2022 and 2023
            if start_time.year == 2021 or end_time.year == 2021:
                # If the sleep segment crosses over to the next day, split it into two segments
                if start_time.day != end_time.day:
                    end_of_day = datetime(start_time.year, start_time.month, start_time.day, 23, 59, 59)
                    start_of_next_day = end_of_day + timedelta(seconds=1)
                    sleep_times_2021.append(((start_time, end_of_day), sleep_duration))
                    sleep_times_2021.append(((start_of_next_day, end_time), sleep_duration))
                else:
                    sleep_times_2021.append(((start_time, end_time), sleep_duration))
                    
            elif start_time.year == 2022 or end_time.year == 2022:
                if start_time.day != end_time.day:
                    end_of_day = datetime(start_time.year, start_time.month, start_time.day, 23, 59, 59)
                    start_of_next_day = end_of_day + timedelta(seconds=1)
                    sleep_times_2022.append(((start_time, end_of_day), sleep_duration))
                    sleep_times_2022.append(((start_of_next_day, end_time), sleep_duration))
                else:
                    sleep_times_2022.append(((start_time, end_time), sleep_duration))
                    
            elif start_time.year == 2023 or end_time.year == 2023:
                if start_time.day != end_time.day:
                    end_of_day = datetime(start_time.year, start_time.month, start_time.day, 23, 59, 59)
                    start_of_next_day = end_of_day + timedelta(seconds=1)
                    sleep_times_2023.append(((start_time, end_of_day), sleep_duration))
                    sleep_times_2023.append(((start_of_next_day, end_time), sleep_duration))
                else:
                    sleep_times_2023.append(((start_time, end_time), sleep_duration))

In [ ]:
def get_sleep_segments(sleep_times, start_year):
    # Create a base timeline for the year
    base_timeline = np.array([datetime(start_year, 1, 1) + timedelta(days=i) for i in range(365)])
    
    # Initialize an array to hold the sleep segments and durations
    sleep_segments = np.empty((365, 24, 3), dtype='O')
    sleep_segments.fill(np.datetime64('1970-01-01T00:00:00'))

    # Loop over the sleep times
    for (start_time, end_time), duration in sleep_times:
        # Find the index of the start time in the base timeline
        start_index = (start_time - datetime(start_year, 1, 1)).days
        if 0 <= start_index < 365:
            # If the sleep segment starts and ends on the same day
            if start_time.day == end_time.day:
                # Find an empty slot for the sleep segment
                for i in range(24):
                    if sleep_segments[start_index, i, 0] == np.datetime64('1970-01-01T00:00:00'):
                        sleep_segments[start_index, i, :] = np.datetime64(start_time), np.datetime64(end_time), duration
                        break
            else:
                # If the sleep segment spans two days, split it at midnight
                sleep_segments[start_index, -1, :] = np.datetime64(start_time), np.datetime64(datetime(start_time.year, start_time.month, start_time.day, 23, 59, 59)), duration
                if start_index+1 < 365:
                    sleep_segments[start_index+1, 0, :] = np.datetime64(datetime(end_time.year, end_time.month, end_time.day, 0, 0, 0)), np.datetime64(end_time), duration
    
    return base_timeline, sleep_segments

In [ ]:
# Get the sleep segments for each year
base_timeline_2021, sleep_segments_2021 = get_sleep_segments(sleep_times_2021, 2021)
base_timeline_2022, sleep_segments_2022 = get_sleep_segments(sleep_times_2022, 2022)
base_timeline_2023, sleep_segments_2023 = get_sleep_segments(sleep_times_2023, 2023)

# Combine the base timelines and sleep segments
base_timeline = np.concatenate([base_timeline_2021, base_timeline_2022, base_timeline_2023])
sleep_segments = np.concatenate([sleep_segments_2021, sleep_segments_2022, sleep_segments_2023])

In [ ]:
# Create the figure and axes with updated aspect ratio
fig, ax = plt.subplots(figsize=(18, 15))

# Set the background color
ax.set_facecolor('#272822')
fig.set_facecolor('#272822')

# Increase the font size for ticks, labels, title slightly
plt.rc('font', size=18)          # controls default text sizes
plt.rc('axes', titlesize=34)     # fontsize of the axes title
plt.rc('axes', labelsize=26)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=24)    # fontsize of the tick labels
plt.rc('ytick', labelsize=18)    # fontsize of the tick labels
plt.rc('legend', fontsize=28)    # legend fontsize
plt.rc('figure', titlesize=34)   # fontsize of the figure title

# Set the color of the ticks, labels, and title
ax.tick_params(colors='white')
ax.xaxis.label.set_color('white')
ax.yaxis.label.set_color('white')
ax.title.set_color('white')

# Loop over the base timeline and sleep segments
for i, (day, day_sleep_segments) in enumerate(zip(base_timeline, sleep_segments)):
    # Sort the sleep segments by start time
    day_sleep_segments = day_sleep_segments[np.argsort(day_sleep_segments[:, 0])]
    # Plot the sleep segments in color if they exist
    for sleep_segment in day_sleep_segments:
        if sleep_segment[0] != np.datetime64('1970-01-01T00:00:00'):
            sleep_start = (sleep_segment[0] - np.datetime64(day)).astype('timedelta64[s]').astype(float) / 86400
            sleep_end = (sleep_segment[1] - np.datetime64(day)).astype('timedelta64[s]').astype(float) / 86400
            sleep_duration = sleep_segment[2]

            # If a segment duration is >4h, draw it green, otherwise draw it purple
            if sleep_duration >= 4:
                line = ax.plot([day, day], [sleep_start, sleep_end], color='#A6E22E', linewidth=2)  # green
            else:
                line = ax.plot([day, day], [sleep_start, sleep_end], color='#AE81FF', linewidth=2)  # purple

# Format the x-axis
ax.xaxis.set_major_locator(mdates.MonthLocator())
ax.xaxis.set_major_formatter(mdates.DateFormatter('%b'))

# Draw the plot
fig.canvas.draw()

# Get the labels
labels = [item.get_text()[0] for item in ax.get_xticklabels()]
ax.set_xticklabels(labels)

# Format the y-axis
ax.set_ylim(0, 1)
ax.set_yticks([i/24 for i in range(1, 24, 2)])
ax.set_yticklabels(['{:02d}:00'.format(i) for i in range(1, 24, 2)], rotation=90, verticalalignment='center')

# Make yticks and ylabel to the right of the plot
ax.yaxis.tick_right()
ax.yaxis.set_label_position("right")

# Set the title and labels
title = ax.set_title(f'{name} fitbit sleep data 2021-12—2023-07', fontsize=34, pad=20)
ax.set_xlabel('Date', labelpad=10)
ax.set_ylabel('Time', labelpad=10)

# Create a legend for colors in the upper right corner
from matplotlib.lines import Line2D
custom_lines = [Line2D([0], [0], color='#A6E22E', lw=2),
                Line2D([0], [0], color='#AE81FF', lw=2)]
legend = ax.legend(custom_lines, ['Sleep (>=4h)', 'Nap (<4h)'], loc='upper right', bbox_to_anchor=(0.95, 0.93), facecolor='#3C3C3C')
plt.setp(legend.get_texts(), color='w') # setting legend text color to white

# Show the plot
plt.tight_layout()
plt.show()